In [1]:
import numpy as np
import dask
import dask.array as da
# %load_ext line_profiler

ImportError: No module named line_profiler

In [3]:
da.transpose

<function numpy.core.fromnumeric.transpose>

In [101]:
def dcg(A, b, tol=1e-8, maxiter=500, x0=None, preconditioner=None, verbose=0, client=None):
    """ Conjugate gradient
        
        Parameters
        ----------
        A: array-like
        b: array-like
        tol: float
        
        Returns
        -------
        x: array-like
        iters: int
        resnorm: float
        
        Find x such that

            Ax = b
        
        for square, symmetric A.

        If a preconditioner M is provided, solve the left-preconditioned
        equivalent problem,

            M(Ax - b) = 0
    """
    print_iter = max(1, maxiter / 10**verbose)

    A, b, M = dask.persist(A, b, preconditioner)

    if x0 is None:
        r = 1 * b
        x = 0 * b
    else:
        r = 1 * b - A.dot(x0)
        x = x0

    Mr = r if M is None else M.dot(r)
    p = Mr
    resnrm2 = r.dot(Mr)

    x, r, p, resnrm2 = dask.persist(x, r, p, resnrm2)
    (resnrm2,) = dask.compute(resnrm2)
    if resnrm2**0.5 < tol:
        return x, 0, resnrm2**0.5

    for k in range(maxiter):
        ox, ores, op, oresnrm2 = x, r, p, resnrm2

        Ap = A.dot(p)
        alpha = resnrm2 / p.dot(Ap)
        x = ox + alpha * p
        r = ores - alpha * Ap
        Mr = r if M is None else M.dot(r)
        resnrm2 = r.dot(Mr)

        x, r, resnrm2 = dask.persist(x, r, resnrm2)
        (resnrm2,) = dask.compute(resnrm2)

        if resnrm2**0.5 < tol:
            break
        elif (k + 1) % print_iter == 0:
            print("ITER: {:5}\t||Ax -  b||_2: {}".format(k + 1, resnrm2**0.5))

        p = Mr + (resnrm2 / oresnrm2) * op
        x, r, resnrm2, p= dask.persist(x, r, resnrm2, p)

        (p,) = dask.persist(p)

    return x, k + 1, resnrm2**0.5


## Numpy array testing

In [87]:
m = 400
n = 300
mc = 100
nc = 100

In [88]:
A = np.random.random((m, n))
x = np.random.random(n)
b = np.random.random(n)
rho = 1.

In [89]:
Asymm_unregularized = A.T.dot(A)

In [90]:
xcg, iters, res = dcg(Asymm_unregularized, b, verbose=5)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:     1	||Ax -  b||_2: 5.4518820281927605
ITER:     2	||Ax -  b||_2: 4.4699918357914825
ITER:     3	||Ax -  b||_2: 3.767801181503383
ITER:     4	||Ax -  b||_2: 3.378497818515491
ITER:     5	||Ax -  b||_2: 2.802273886853752
ITER:     6	||Ax -  b||_2: 2.358055618490828
ITER:     7	||Ax -  b||_2: 8.326831181577527
ITER:     8	||Ax -  b||_2: 1.937614759852169
ITER:     9	||Ax -  b||_2: 1.779677110707361
ITER:    10	||Ax -  b||_2: 1.5640713788686087
ITER:    11	||Ax -  b||_2: 1.2515824824691368
ITER:    12	||Ax -  b||_2: 1.0224270793109556
ITER:    13	||Ax -  b||_2: 15.17339267627602
ITER:    14	||Ax -  b||_2: 0.856835348069205
ITER:    15	||Ax -  b||_2: 0.7750949007734822
ITER:    16	||Ax -  b||_2: 0.6278138729140018
ITER:    17	||Ax -  b||_2: 0.4930684387451341
ITER:    18	||Ax -  b||_2: 0.40511716324915664
ITER:    19	||Ax -  b||_2: 3.820740754413606
ITER:    20	||Ax -  b||_2: 0.3368478904826705
ITER:    21	||Ax -  b||_2: 0.2813460841866662
ITER:    22	||Ax -  b||_2: 0.23120761397106

In [8]:
Asymm_weakreg = A.T.dot(A) + 0.1 * np.eye(n)

In [9]:
xcg, iters, res = dcg(Asymm_weakreg, b)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:  132
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.68109585745026e-10


In [10]:
Asymm = A.T.dot(A) + np.eye(n)

In [11]:
xcg, iters, res = dcg(Asymm, b, verbose=1)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITER:    50	||Ax -  b||_2: 0.0002016569187775861
ITERS:   93
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.63327173186293e-10


# Preconditioning

In [12]:
xcg, iters, res = dcg(Asymm, b, preconditioner=np.eye(n))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   93
||(rho * I + A'A)x - b||_2 / sqrt(n): 4.63327173186293e-10


In [13]:
class JacobiPrecond(object):
    """ Build a Jacobi diagonal preconditioner P = inv(diag(A))
    """
    def __init__(self, A_symmetric):
        self.d = np.reciprocal(np.diag(A_symmetric))
    def dot(self, v):
        return self.d * v

In [14]:
xcg, iters, res = dcg(Asymm, b, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   82
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.355255222025486e-10


## Warm start
Let $x_{cg}$ be the solution to $Ax = b$, then set $b_{new} = b + b'$ for some (small) perturbation $b'$, and re-solve the system with and without providing $x_{cg}$ as an initial guess. 

We also compare the effects of warm starting with those of using a preconditioner.

In [16]:
pct_perturb = 5.
perturb = np.random.random(n) - 0.5
perturb = (perturb / np.linalg.norm(perturb)) * (pct_perturb / 100.) * np.linalg.norm(b) 
b_new = b + perturb

In [17]:
xwarm, iters, res = dcg(Asymm, b_new, x0=xcg, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   72
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.742118667763169e-10


In [18]:
xcold, iters, res = dcg(Asymm, b_new, preconditioner=JacobiPrecond(Asymm))
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   82
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.417050277638157e-10


In [19]:
xwarm_noprecon, iters, res = dcg(Asymm, b_new, x0=xcg)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   84
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.503911726230131e-10


In [20]:
xcold_noprecon, iters, res = dcg(Asymm, b_new)
print("ITERS: {:4}".format(iters))
print("||(rho * I + A'A)x - b||_2 / sqrt(n): {}".format(res / n**0.5))

ITERS:   93
||(rho * I + A'A)x - b||_2 / sqrt(n): 5.55860949899645e-10


|Conditions                  | Iterations |
|:---------------------------|------------|
|Cold start                  | 93         |
|Cold start + preconditioner | 84         |
|Warm start                  | 82         |
|Warm start + preconditioner | 72         |

## Dask vs. numpy arrays

In [21]:
c = n/2

In [22]:
Ad = da.from_array(Asymm, chunks=c)
bd = da.from_array(b, chunks=c)

In [23]:
%lprun -f dcg dcg(Ad, b)

In [24]:
%lprun -f dcg dcg(Asymm, b)

For $A\in R^{300\times 300}$, the CG runtime is around 6ms using ``numpy`` arrays. When we switch to ``dask`` arrays, the results (solution $x$, iteration count $k$, and residual $\|Ax-b\|_2$) are all the same, but the call now takes around 2s.